# Train Diffusion model

> Training code for diffusion model on cifar

In [ ]:
# | default_exp train.cifar

In [ ]:
# | hide
from nbdev.showdoc import *

%reload_ext jupyternotify
%reload_ext nb_black
%reload_ext autoreload
%reload_ext line_profiler
%autoreload 2
%env CUDA_VISIBLE_DEVICES=

import sys

__root = "../../"
sys.path.append(__root)

In [ ]:
# | export
from torch_snippets import *
from simple_ddpm.data.load_cifar import *
from simple_ddpm.model.ddpm import *

In [ ]:
from simple_ddpm.config.cifar import Config
from simple_ddpm.data.load_cifar import CIFARCars

config = Config()
config.BATCH_SIZE = 8
data = CIFARCars("/Users/yeshwanth.y/Downloads/data", config)
dl = data.make_dataloader(collate_function=data.generate_noise_collator)
batch = next(iter(dl))

In [ ]:
from simple_ddpm.config.cifar import Config

config = Config()
model = Model(config)
model.show_stepwise_denoise(n=2)

In [ ]:
from transformers import TrainingArguments, Trainer

train_dataset = data.data

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    max_steps=100,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="steps",
    save_steps=10,
    eval_steps=10,
    logging_steps=10,
    logging_dir="./logs",
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data.generate_noise_collator,
    train_dataset=train_dataset,
    eval_dataset=torch.utils.data.Subset(data.data, [1, 2, 3, 4]),
)

# Train the model
trainer.train()

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()